# Containers scheduling
[![containers_scheduling.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/marcos-dv/scheduling/containers_scheduling.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/marcos-dv/scheduling/containers_scheduling.ipynb) [![Open In Deepnote](https://deepnote.com/buttons/launch-in-deepnote-small.svg)](https://deepnote.com/launch?url=https://github.com/ampl/colab.ampl.com/blob/master/authors/marcos-dv/scheduling/containers_scheduling.ipynb) [![Open In Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/marcos-dv/scheduling/containers_scheduling.ipynb) [![Open In Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/marcos-dv/scheduling/containers_scheduling.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/marcos-dv/scheduling/containers_scheduling.ipynb) [![Powered by AMPL](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/marcos-dv/scheduling/containers_scheduling.ipynb)](https://ampl.com)

Description: Scheduling model for harbor operations. It is a problem with dependences between containers, which should be dispatch the fastest possible. We are using the MP solver interfaces to model a complex system using techniques from Constraint Programming, such as indicator constraints, and logical or and forall operators. After the model is written, a couple instances are presented and Highs/Gurobi MIP solvers are used to tackle the problem.

Tags: amplpy, scheduling, industry, mip, constraint-programming, mp

Notebook author: Marcos Dominguez Velad <<marcos@ampl.com>>

In [1]:
# Install dependencies
%pip install -q amplpy

In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs", "gurobi"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

## Containers scheduling

The harbor wants to dispath the most containers possible, so they can be shipped soon.

The container scheduling problem is a general scheduling problem where we want to move some containers (tasks) with some cranes (the agents). However, containers are dependent between themselves, as they are grouped in stacks. Then, we can't dispatch a container until we have dispatched every container above it.

* There are work periods / shifts in which machines can work.
* Multiple stacks of containers
* Different cranes


## Model

### Sets and parameters

* Set of containers $C$, each container is descripted by 2 coordinates: $(i,c)$. $i$ is the stack of the container, and $c$ the order inside the stack. Blocks/stacks of containers are independent, but 2 machines can be working on the same stack during a period of time.

<div>
    <img src="https://raw.githubusercontent.com/ampl/colab.ampl.com/master/authors/marcos-dv/scheduling/imgs/containers3.png" width="500"/>
</div>

* Set of periods of time $T$. These periods are assume to be equal.

* Set of machines $M$. Each machine has a work capacity to move the containers.

* To dispatch a container, we have to work at least the move cost of a container. 
  

### Variables

* $c\_moving_{m,i,c,t}$: binary variable equals 1 if machine $m$ is working on container $(i,c)$ in period $t$, 0 otherwise.

* $c\_end_{i,c,t}$: binary variable equals 1 if the container $(i,c)$ is already dispatched in period $t$, 0 otherwise. If we finish a container in time $t$, then $c\_end$ should be 1, although it wasn't finished at the beginning of the time period.

* $c\_progress_{m,i,c,t}$: continuous $\geq 0$ variable that saves the amount of work we did on container $(i,c)$ in period $t$ with machine $m$.

The rest of the model are constraints that give meaning and bounds to these variables. We are using some Constraint Programming operators to write the model efficiently.



## Constraints

* Maximum progress of container $(i,c)$:

$$
	\sum \limits_{m \in M, t \in T} c\_progress_{m,i,c,t} \leq move\_cost_{i,c}
$$

in ampl:
```
subject to progress_limit_container{(i,c) in C}:
	sum{m in M, t in T} c_progress[m,i,c,t] <= move_cost[i,c];
```

* Maximum work of machine $m$ in time $t$:

$$
	\sum \limits_{(i,c) \in C} c\_progress_{m,i,c,t} \leq work\_cap_m
$$

in ampl:
```
subject to progress_limit{m in M, t in T}:
	sum{(i,c) in C} c_progress[m,i,c,t] <= work_cap[m];
```

Bonds between variables...

* Bond between move and progress:

$$
	c\_mov_{m,i,c,t} = 1 \iff c\_progress_{m,i,c,t} > 0
$$

in ampl:
```
subject to active_work{m in M, (i,c) in C, t in T}:
	c_moving[m,i,c,t] = 1 <==> c_progress[m,i,c,t] > 0;
```
(logical constraints are linearize through the [MP solver](https://mp.ampl.com/index.html) interfaces)

* Bond between end a container and progress:

$$
	c\_end_{i,c,t} = 1 \iff \sum \limits_{m \in M, \; t' = 1}^{t} c\_progress_{m,i,c,t'} = move\_cost_{i,c}
$$

```
subject to finish{(i,c) in C, t in T}:
	c_end[i,c,t] = 1 <==> sum{m in M, t_ in 1..t} c_progress[m,i,c,t_] >= move_cost[i,c];
```

Containers logic constraints...

* Containers are in a stack:

$$
	c\_mov_{m,i,c,t} \leq c\_end_{i,c-1,t}\;, \quad \forall m \in M, (i,c) \in C: \ c \neq 1, \; t \in T
$$

in ampl:
```
subject to previous_container{m in M, (i,c) in C, t in T: c > 1}:
	c_moving[m,i,c,t] <= c_end[i,c-1,t];
```

* A finished container doesn't move:

$$
	c\_end_{\, i,c,t} = 1 \implies c\_mov_{m,i,c,t+1} = 0
$$

in ampl:
```
subject to already_finished{m in M, (i,c) in C, t in T: t <> last(T)}:
	c_end[i,c,t] = 1 ==> c_moving[m,i,c,t+1] = 0;
```

* We need to move every container.

$$
	c\_end_{\, i,c,last(T)} = 1, \quad \forall (i,c) \in C
$$

in ampl:
```
subject to finish_all{(i,c) in C}:
	c_end[i,c,last(T)] = 1;
```

* $c_end$ variable is monotonous, so we can write some constraints to strengthen the model so it gets solved faster.

$$
	c\_end_{\, i,c,t-1} \leq c\_end_{\, i,c,t}, \quad \forall (i,c) \in C, \ \forall t \in T \setminus \{ 1 \}
$$

$$
	c\_end_{\, i,c-1,t} \geq c\_end_{\, i,c,t}, \quad \forall (i,c) \in C: \ c \neq 1 \ \forall t \in T
$$

in ampl:
```
 c_end[i,c,t-1] <= c_end[i,c,t];
 c_end[i,c-1,t] >= c_end[i,c,t];
```

* If a machine is moving a block, no other machine can use it.

$$
	\sum \limits_{(i',c) \in C: \ i = i'} c\_mov_{i,c,t} \geq 1 \implies \forall_{\substack{(i', c) \in C: \ i != i'}} \ c\_mov_{i',c,t} = 0
$$

in ampl:
```
subject to only_one_block{m in M, i in B, t in T}:
	sum{(i_,c) in C: i == i_} c_moving[m,i,c,t] >= 1 ==> forall{(i_, c) in C: i != i_} c_moving[m,i_,c,t] = 0;
```

* Finish the partially moved containers.

$$
	c\_mov_{m,i,c,t-1} = 1 \implies c\_mov_{m,i,c,t} = 1 \lor c\_end_{i,c,t-1} = 1
$$

in ampl:
```
subject to cant_forget_active_containers{m in M, (i,c) in C, t in T: t != last(T)}:
	c_moving[m,i,c,t] = 1 ==> c_moving[m,i,c,t+1] = 1 or c_end[i,c,t] = 1;
```

* A container is only dispatched by 1 machine:

$$
	\sum \limits_{m \in M} c\_mov_{m,i,c,t} \leq 1
$$

in ampl:
```
subject to one_mach_per_container{(i,c) in C, t in T}:
	sum{m in M} c_moving[m,i,c,t] <= 1;
```

## Objective

We have no objective in this problem, but if we can add some of them to get "better" feasible solutions. For example, giving some penalties to delayed blocks or unused capacity of the machines.

$$
    \min \quad delay\_penalty + unused\_cap\_penalty
$$

where

$$
    delay\_penalty = \sum \limits_{(i,c) \in C, \; t \in T} (1-c\_end_{\, i,c,t })
$$

$$
    unused\_cap\_penalty = \sum \limits_{m \in M, \; (i,c) \in C, \; t \in T} \frac{t}{|T|} \cdot \frac{c\_progress_{m,i,c,t}}{total\_move}
$$

in ampl:
```
var delay_penalty = sum{(i,c) in C, t in T} (1-c_end[i,c,t]);
var unused_cap_penalty = sum{m in M, (i,c) in C, t in T} t/card(T)*c_progress[m,i,c,t]/total_move;
minimize penalty: unused_cap_penalty + delay_penalty;
```

With this formulation, we get a neat model easy to read despite of being a complex system.

We are using many logical constraints such as indicator constraints, logical operators (`or` and `forall`), but we can still use a MIP solver such as Highs or Gurobi to solve the problem due to the driver's reformulation.

In [3]:
%%ampl_eval

reset;

# Blocks
set B;
# Time periods
set T ordered;
# Containers
set C dimen 2; # (block, container)
# Machines
set M;

# Cost per container
param move_cost{C};
param total_move := sum{(i,c) in C} move_cost[i,c];

# Work capacity of the crane
param work_cap {M};

# Vars #

# Container in progress, container is finished, container progress units
var c_moving{M,C,T} binary;
var c_end{C,T} binary;
var c_progress{m in M,C,T} >= 0 <= work_cap[m];

# Constraints #

## Container completion

# Cant progress more than available in the container
subject to progress_limit_container{(i,c) in C}:
	sum{m in M, t in T} c_progress[m,i,c,t] <= move_cost[i,c];

# Cant progress more than available by the machine
subject to progress_limit{m in M, t in T}:
	sum{(i,c) in C} c_progress[m,i,c,t] <= work_cap[m];

## Containers logic

# Active iff in progress:
subject to active_work{m in M, (i,c) in C, t in T}:
	c_moving[m,i,c,t] = 1 <==> c_progress[m,i,c,t] > 0;
	# c_moving[m,i,c,t] = 1 <==> c_progress[m,i,c,t] >= 1e-9;

# if completely moved then finish container.
subject to finish{(i,c) in C, t in T}:
	c_end[i,c,t] = 1 <==> sum{m in M, t_ in 1..t} c_progress[m,i,c,t_] >= move_cost[i,c];

# A container is not active unless previous has finished:
subject to previous_container{m in M, (i,c) in C, t in T: c > 1}:
	c_moving[m,i,c,t] <= c_end[i,c-1,t];

# A container is not active if it already finished:
subject to already_finished{m in M, (i,c) in C, t in T: t <> last(T)}:
	c_end[i,c,t] = 1 ==> c_moving[m,i,c,t+1] = 0;

subject to finish_all{(i,c) in C}:
	c_end[i,c,last(T)] = 1;

# Finished container can't go unfinished
subject to remain_finished{(i,c) in C, t in T: t <> last(T)}:
	c_end[i,c,t] <= c_end[i,c,t+1];

# Cant go to other blocks
subject to only_one_block{m in M, i in B, t in T}:
	sum{(i_,c) in C: i == i_} c_moving[m,i,c,t] >= 1 ==> forall{(i_, c) in C: i != i_} c_moving[m,i_,c,t] = 0;

# Cant forget containers
subject to cant_forget_active_containers{m in M, (i,c) in C, t in T: t != last(T)}:
	c_moving[m,i,c,t] = 1 ==> c_moving[m,i,c,t+1] = 1 or c_end[i,c,t] = 1;

# One machine per container atmost
subject to one_mach_per_container{(i,c) in C, t in T}:
	sum{m in M} c_moving[m,i,c,t] <= 1;

# Objective #
var delay_penalty = sum{(i,c) in C, t in T} (1-c_end[i,c,t]);
#var delay_penalty_scaled = sum{c in C, t in T} (1-c_end[c,t])/card({C,T});
# Scaled!!
var unused_cap_penalty = sum{m in M, (i,c) in C, t in T} t/card(T)*c_progress[m,i,c,t]/total_move;
minimize penalty: unused_cap_penalty + delay_penalty;

## Solving a simple instance

We are using the open source solver highs to solve an instance of the problem:

In [4]:
def container_cost(i, l):
    return max(3, (i + l) % 7)


periods = 10
containers = 4
blocks = 2
machines = 2
container_costs = {
    (i, l): container_cost(i, l)
    for i in range(1, blocks + 1)
    for l in range(1, containers + 1)
}
print(container_costs)
work_capacity = {m: 5 for m in range(1, machines + 1)}

ampl.set["T"] = range(1, periods + 1)
ampl.set["B"] = range(1, blocks + 1)
ampl.set["M"] = range(1, machines + 1)
ampl.set["C"] = [(i, l) for i in range(1, blocks + 1) for l in range(1, containers + 1)]
ampl.param["move_cost"] = container_costs
ampl.param["work_cap"] = work_capacity

ampl.solve(solver="highs")
assert ampl.solve_result == "solved", ampl.solve_result

# ampl.display('c_moving')
# ampl.display('c_end')
ampl.display(
    "{m in M, (i,c) in C, t in T : c_progress[m,i,c,t] >= 1e-6} c_progress[m,i,c,t]"
)

{(1, 1): 3, (1, 2): 3, (1, 3): 4, (1, 4): 5, (2, 1): 3, (2, 2): 4, (2, 3): 5, (2, 4): 6}
HiGHS 1.8.1: optimal solution; objective 10.21818182
7327 simplex iterations
22 branching nodes
absmipgap=3.55271e-15, relmipgap=0
c_progress[m,i,c,t] :=
1 2 1 1   3
1 2 2 1   2
1 2 2 2   2
1 2 3 2   3
1 2 3 3   2
1 2 4 3   3
1 2 4 4   3
2 1 1 1   3
2 1 2 1   2
2 1 2 2   1
2 1 3 2   4
2 1 4 3   5
;



## Solving a more difficult instance

We are using Gurobi, a powerful commercial solver to solve the instance. Notice that we are using `timeout=150s` so Gurobi only runs for 150 seconds, which is enough for giving a good solution.

We can check some stats of the solving process through the `outlev=1` option.

(If you are using a Community Edition from Google Colab, you can change to the 'highs' solver and try to solve the instance)

In [5]:
def container_cost(i, l):
    return max(3, (i + l * 17) % 10)


periods = 20
containers = 8
blocks = 4
machines = 3
container_costs = {
    (i, l): container_cost(i, l)
    for i in range(1, blocks + 1)
    for l in range(1, containers + 1)
}
work_capacity = {m: 5 + m for m in range(1, machines + 1)}

ampl.set["T"] = range(1, periods + 1)
ampl.set["B"] = range(1, blocks + 1)
ampl.set["M"] = range(1, machines + 1)
ampl.set["C"] = [(i, l) for i in range(1, blocks + 1) for l in range(1, containers + 1)]
ampl.param["move_cost"] = container_costs
ampl.param["work_cap"] = work_capacity

ampl.solve(solver="gurobi", gurobi_options="outlev=1 timelim=150")
assert ampl.solve_result in ["solved", "limit"], ampl.solve_result

# ampl.display('c_moving')
# ampl.display('c_end')
ampl.display(
    "{m in M, (i,c) in C, t in T : c_progress[m,i,c,t] >= 1e-6} c_progress[m,i,c,t]"
)

Gurobi 12.0.0: Set parameter LogToConsole to value 1
  tech:outlev = 1
Set parameter TimeLimit to value 150
  lim:time = 150
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  150
InfUnbdInfo  1

Optimize a model with 8232 rows, 18465 columns and 22616 nonzeros
Model fingerprint: 0x836230f6
Model has 9424 simple general constraints
  240 AND, 3888 OR, 5296 INDICATOR
Variable types: 1921 continuous, 16544 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-04, 1e+00]
  Bounds range     [1e+00, 6e+02]
  RHS range        [1e+00, 9e+00]
  GenCon rhs range [1e-04, 9e+00]
  GenCon coe range [1e+00, 1e+00]

User MIP start did not produce a new incumbent solution
User MI